# SpatialLIBD

## 1. Download all samples from SpatialLIBD

In [1]:
library(glue)

In [2]:
# Prepare directory names
main_data_dir_name <- "data"
dataset_dir_name <- "SpatialLIBD"
dataset_images_dir_name <- "images"
dataset_h5ad_dir_name <- "h5ad"
dataset_dir_path <- file.path(main_data_dir_name, dataset_dir_name)
dataset_images_dir_path <- file.path(dataset_dir_path, dataset_images_dir_name)
dataset_r_object_name <- "spe.rds"
dataset_r_object_path <- file.path(dataset_dir_path, dataset_r_object_name)
dataset_h5ad_dir_path <- file.path(dataset_dir_path, dataset_h5ad_dir_name)
print(dataset_r_object_path)

[1] "data/SpatialLIBD/spe.rds"


In [3]:
# Check if parent data directory and dataset directory exists. If not, create.
if (!file.exists(main_data_dir_name))
    dir.create(main_data_dir_name)
if (!file.exists(dataset_dir_path))
    dir.create(dataset_dir_path)

In [4]:
# If dataset object file does not exist, download the object and save it.
if (!file.exists(dataset_dir_path)) {
    if (!requireNamespace("BiocManager", quietly = TRUE))
        install.packages("BiocManager")

    if (!requireNamespace("spatialLIBD", quietly = TRUE))
        BiocManager::install("spatialLIBD")

    library("spatialLIBD")

    # set timeout to 10 minutes because simultaneous download of samples leads to timeout
    # default: 60 seconds
    options(timeout = 600)

    spe <- fetch_data(type = "spe")

    # set timeout back to default value
    options(timeout = 60)

    # save object to file
    saveRDS(spe, file=dataset_r_object_path)
    
} else {
    spe <- readRDS(dataset_r_object_path)
}


In [5]:
spe

Loading required package: SpatialExperiment

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, 

class: SpatialExperiment 
dim: 33538 47681 
metadata(0):
assays(2): counts logcounts
rownames(33538): ENSG00000243485 ENSG00000237613 ... ENSG00000277475
  ENSG00000268674
rowData names(9): source type ... gene_search is_top_hvg
colnames(47681): AAACAACGAATAGTTC-1 AAACAAGTATCTCCCA-1 ...
  TTGTTTCCATACAACT-1 TTGTTTGTGTAAATTC-1
colData names(69): sample_id Cluster ... array_row array_col
reducedDimNames(6): PCA TSNE_perplexity50 ... TSNE_perplexity80
  UMAP_neighbors15
mainExpName: NULL
altExpNames(0):
spatialCoords names(2) : pxl_col_in_fullres pxl_row_in_fullres
imgData names(4): sample_id image_id data scaleFactor

In [6]:
sample_ids <- unique(colData(spe)[['sample_id']])

In [7]:
# Download images if they are not yet available
if (!file.exists(dataset_images_dir_path)) {
    dir.create(dataset_images_dir_path)
    downloadImage <- function(sample_id) {
        image_url <- imgSource(samples[[sample_id]])
        dest_file <- file.path(dataset_images_dir_path, glue("{sample_id}.png"))
        curl::curl_download(
            url = image_url, 
            destfile = dest_file)
    }
    lapply(sample_ids, downloadImage)
}

In [8]:
# Separate spatial experiment object into samples
samples <- lapply(sample_ids, function(sample_id) {
    spe[,spe$sample_id == sample_id]
})

In [9]:
# Attach names to samples
names(samples) <- sample_ids

In [16]:
samples[[1]]

class: SpatialExperiment 
dim: 33538 4226 
metadata(0):
assays(2): counts logcounts
rownames(33538): ENSG00000243485 ENSG00000237613 ... ENSG00000277475
  ENSG00000268674
rowData names(9): source type ... gene_search is_top_hvg
colnames(4226): AAACAACGAATAGTTC-1 AAACAAGTATCTCCCA-1 ...
  TTGTTTCCATACAACT-1 TTGTTTGTGTAAATTC-1
colData names(69): sample_id Cluster ... array_row array_col
reducedDimNames(6): PCA TSNE_perplexity50 ... TSNE_perplexity80
  UMAP_neighbors15
mainExpName: NULL
altExpNames(0):
spatialCoords names(2) : pxl_col_in_fullres pxl_row_in_fullres
imgData names(4): sample_id image_id data scaleFactor

In [10]:
if (!file.exists(dataset_h5ad_dir_path)) {
    BiocManager::install("zellkonverter")
    
    # Next step: save as H5AD files
    library(zellkonverter)
    library(basilisk)
    library(reticulate)
        
    # attach coordinates of spots before saving as annData
    for (sample_id in sample_ids) {
        colData(samples[[sample_id]]) <- cbind(colData(samples[[sample_id]]), spatialCoords(samples[[sample_id]]))
    }

    dir.create(dataset_h5ad_dir_path)

    zellkonverterEnvironment <- zellkonverterAnnDataEnv()
    
    # Function from
    # https://github.com/LieberInstitute/spatialDLPFC/blob/14a1f253a92e43c01fec3cc3077a9b2cf9ce9fc0/code/spython/spagcn/code/02-our_data_tutorial/01-spe_to_anndata.R#L28
    write_anndata <- function(sce, out_path) {
        invisible(basiliskRun(
            fun = function(sce, filename) {
                # Convert SCE to AnnData:
                adata <- SCE2AnnData(sce)

                # Write AnnData object to disk
                adata$write(filename = filename)
            },
            env = zellkonverterEnvironment,
            sce = sce,
            filename = out_path
        ))
    }

    lapply(sample_ids, function (sample_id) {
        h5ad_file_path <- file.path(dataset_h5ad_dir_path, glue("{sample_id}.h5ad"))
        write_anndata(samples[[sample_id]], h5ad_file_path)
    })
}

In [11]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/edgar/miniconda3/envs/STanndata/lib/libopenblasp-r0.3.23.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] SpatialExperiment_1.8.0     SingleCellExperiment_1.20.0
 [3] SummarizedExperiment_1.28.0 Biobase_2.58.0             
 [5] GenomicRanges_1.50.0        GenomeInfoDb_1.34.9        
 [7] IRanges_2.32.0              S4Vectors_0.36.0           
 [9] BiocGenerics_0.44.0         MatrixGenerics_1.10.0      
[11] matrixStats_1.2.0           glue_1.7